In [5]:
import requests
import pandas as pd
import time
import re
import csv
import os
from bs4 import BeautifulSoup
import random
import pickle
from tqdm import tqdm
import numpy as np 

pd.set_option("display.max_columns", None) # 表示する列数の制限を解除

import sys
sys.path.append("..") # 親ディレクトリを追加
from module.path_reader import PathReader
from module.get_html import GetHTML

In [6]:
# what: HTMLを解析して馬の結果テーブルをDataFrame化する関数
# for:  特徴量抽出のため
# in:   取得したhtml(.bin)
# out:  レース結果テーブル(DataFrame)
def parse_horse_html(bin_path):
    with open(bin_path, "rb") as f:
            html_text = f.read().decode("EUC-JP", errors="ignore")
    soup = BeautifulSoup(html_text, "html.parser")

    # --- レース成績表の抽出 ---
    result_table = soup.find("table", class_="db_h_race_results")
    
    if not result_table:
        raise ValueError("馬結果テーブルが見つかりません。")

    rows = result_table.find_all("tr")[1:]  # ヘッダを除外して行ごとにデータを取得
    horse_data = []

    for row in rows:
        cols = row.find_all("td")
        # --- 日付を取得 ---
        date_tag = row.find("a", href=re.compile(r"/race/list/(\d+)"))
        race_date = re.search(r"/race/list/(\d+)", date_tag["href"]).group(1) if date_tag else None

        # --- race idを取得 ---
        race_id_tag = row.find("a", href=re.compile(r"/race/(\d+)"))
        race_id = re.search(r"/race/(\d+)", race_id_tag["href"]).group(1) if race_id_tag else None

        # --- jockey idを取得 ---
        jockey_id_tag = row.find("a", href=re.compile(r"/jockey/result/recent/(\d+)"))
        jockey_id = re.search(r"/jockey/result/recent/(\d+)", jockey_id_tag["href"]).group(1) if jockey_id_tag else None

        horse_data.append([
            race_date,                     # レース日付
            cols[1].get_text(strip=True),  # 開催
            cols[2].get_text(strip=True),  # 天気
            cols[3].get_text(strip=True),  # R
            race_id,                       # race_id
            cols[6].get_text(strip=True),  # 頭数
            cols[7].get_text(strip=True),  # 枠番
            cols[8].get_text(strip=True),  # 馬番
            cols[9].get_text(strip=True),  # オッズ
            cols[10].get_text(strip=True), # 人気
            cols[11].get_text(strip=True), # 着順
            jockey_id,                     # 騎手
            cols[13].get_text(strip=True), # 斤量
            cols[14].get_text(strip=True), # 距離
            cols[16].get_text(strip=True), # 馬場
            cols[18].get_text(strip=True), # タイム
            cols[19].get_text(strip=True), # 着差
            cols[21].get_text(strip=True), # 通過
            cols[22].get_text(strip=True), # ペース
            cols[23].get_text(strip=True), # 上り
            cols[24].get_text(strip=True), # 馬体重
            # cols[27].get_text(strip=True), # 勝ち馬
            cols[28].get_text(strip=True), # 賞金
        ])
    horse_df = pd.DataFrame(horse_data, columns=[
        "race_date", "place", "weather", "race_no", "race_id", "num_of_horses", "wakuban", "umaban",
        "odds", "popularity", "finish_position", "jockey_id", "weight_carried", "distance", "course_condition",
        "time", "margin", "passing", "pace", "final_3f", "weight", "prize_money"
    ])

    return horse_df

In [7]:
# what: 馬結果テーブルの前処理をする関数
# for:  AIモデルがうけつけられるようにする
# in:   レース結果テーブルの列(.pkl)
# out:  レース結果テーブルの列(.pkl)

def encoding_weather(weather):
    # {晴:0.0, 曇:1.0, 雨:2.0, 小雨:3.0, 雪:4.0, その他:np.nan}
    if pd.isna(weather): 
        return np.nan
    weather_char = str(weather)
    if weather_char == "晴":
        return 0.0
    elif weather_char == "曇":
        return 1.0
    elif weather_char == "小雨":
        return 2.0
    elif weather_char == "雨":
        return 3.0
    elif weather_char == "雪":
        return 4.0
    else:
        return np.nan

def parse_encoding_distance(distance):
    # 例: "芝1800" -> (0.0, 1800)
    # {芝:0.0, ダ:1.0, 障: 2.0, その他:np.nan}
    if pd.isna(distance): 
        return (np.nan, np.nan)
    
    # --- 馬場(state) ---
    state_char = str(distance[0])
    if state_char == "芝":
        state = 0.0
    elif state_char == "ダ":
        state = 1.0
    elif state_char == "障":
        state = 2.0
    else:
        state = np.nan
    
    # --- 距離(length) ---
    try:
        long = float(distance[1:])
    except:
        long = np.nan

    return (state, long)

def encoding_course_condition(condition):
    # {良:0.0, 稍重:1.0, 重:2.0, 不良:3.0, その他:np.nan}
    if pd.isna(condition): 
        return np.nan
    condition_char = str(condition)
    if condition_char == "良":
        return 0.0
    elif condition_char == "稍":
        return 1.0
    elif condition_char == "重":
        return 2.0
    elif condition_char == "不良":
        return 3.0
    else:
        return np.nan

def time_to_seconds(tstr):
    # "1:51.3" -> seconds float
    try:
        if pd.isna(tstr): return np.nan
        if ":" in str(tstr):
            mm, ss = str(tstr).split(":")
            return int(mm) * 60 + float(ss)
        else:
            return float(tstr)
    except:
        return np.nan
    
def parse_margin(margin):
    # 着差の文字列を馬身(float)に変換する関数
    if pd.isna(margin):
        return np.nan
    s = str(margin).strip()
    if s == "":
        return np.nan
    # 正規化（全角スペースや「馬身」などの語を除去）
    s = s.replace("　", " ").replace("馬身", "").replace("馬", "").strip()

    # 特殊語のマッピング（単位は「馬身」）
    special = {"大差": 10.0, "着差": np.nan, "鼻": 0.05, "ハナ": 0.05, "アタマ": 0.1, "クビ": 0.25, "短": 0.05}
    if s in special:
        return special[s]

    # 「1 1/2」や「1-1/2」等の分数表現
    try:
        # 全体と分数（例: "1 1/2" -> whole=1, frac="1/2"）
        if " " in s and "/" in s:
            whole, frac = s.split()
            num, den = frac.split("/")
            return float(whole) + float(num) / float(den)
        if "-" in s and "/" in s:
            whole, frac = s.split("-")
            num, den = frac.split("/")
            return float(whole) + float(num) / float(den)
        if "/" in s and not any(c.isalpha() for c in s):
            num, den = s.split("/")
            return float(num) / float(den)
    except Exception:
        pass

    # 小数や整数に直接変換できればそのまま馬身として返す
    try:
        return float(s)
    except Exception:
        pass

    # その他は欠損とする
    return np.nan

def parse_passing(passing):
    # "3-4-4-3" -> [3,4,4,3]
    passing_str = str(passing)
    if not isinstance(passing_str, str) or passing_str.strip() == "":
        return [np.nan, np.nan, np.nan, np.nan]

    parts = passing_str.split("-")

    result = []
    for i in range(4):
        if i < len(parts) and parts[i].isdigit():
            result.append(float(parts[i]))
        else:
            result.append(np.nan)
    return result

def parse_pace(pace):
    # "34.1-44.8" -> [34.4,44.8]
    try:
        if pd.isna(pace): return [np.nan, np.nan]
        parts = str(pace).split("-")
        return [float(p) for p in parts]
    except:
        return [np.nan, np.nan]

def parse_bodyweight(bw):
    # "494(-4)" -> weight=494, diff=-4
    try:
        s = str(bw)
        if "(" in s:
            w = float(s.split("(")[0])
            diff = float(s.split("(")[1].rstrip(")"))
        else:
            w = float(s)
            diff = np.nan
        return (w, diff)
    except:
        return (np.nan, np.nan)

## 実行関数

In [12]:
# 入力パラメータ

# 実行環境(NotePC/Desktop)の選択
# reader = PathReader("../file_path_NotePC.json") # NotePC用
reader = PathReader("../file_path_Desktop.json") # Desktop用

race_result_pkl = "race_result_table.pkl" # レース結果のテーブル(.pkl)
horse_list_csv = "horse_id_list.csv" # 馬IDリスト(.csv)
horse_result_pkl = "horse_result_table.pkl" # 馬の過去成績テーブル(.pkl)

# # trainer_idとjockey_idのリストも作成
# df_trainer_id = df["trainer_id"].unique().tolist()
# df_jockey_id = df["jockey_id"].unique().tolist()


In [10]:
# データフォルダのインスタンス実体化
data_folder = reader.get_path("data_folder")

# race_resultテーブルを読み込みhorse_idのみを抽出
df = pd.read_pickle(os.path.join(data_folder, race_result_pkl)) # race_resultテーブルの読み出し
horse_id_list = pd.Series(df["horse_id"].unique()) # horse_idのリスト(DataFrame)

# 取得したhorse_idをcsvに保存
horse_id_list.to_csv(os.path.join(data_folder, horse_list_csv), index=False, header=False)

# # trainer_idとjockey_idのリストも作成
# df_trainer_id = df["trainer_id"].unique().tolist()
# df_jockey_id = df["jockey_id"].unique().tolist()

### 各馬の成績ページ(HTML)をbinファイルとして保存

In [13]:
# CSVの読み込み
df_horse = pd.read_csv(os.path.join(data_folder, horse_list_csv), header=None)
horse_ids = df_horse[0].astype(str).tolist()
get_html = GetHTML()

for horse_id in tqdm(horse_ids, total=len(horse_ids)):
    url = f"https://db.netkeiba.com/horse/result/{horse_id}" # 取得先URL
    save_path = os.path.join(data_folder, "horse_result_html", f"{horse_id}.bin") # ファイル保存パス 
    get_html.get_and_save(url, save_path) # HTMLの取得と保存

  0%|          | 0/13118 [00:00<?, ?it/s]

[Skip] Existing bin file: C:\Users\yasak\Desktop\mykeibaAI\data\horse_result_html\2018100377.bin
[Skip] Existing bin file: C:\Users\yasak\Desktop\mykeibaAI\data\horse_result_html\2018102969.bin
[Skip] Existing bin file: C:\Users\yasak\Desktop\mykeibaAI\data\horse_result_html\2017106394.bin
[Skip] Existing bin file: C:\Users\yasak\Desktop\mykeibaAI\data\horse_result_html\2017101726.bin
[Skip] Existing bin file: C:\Users\yasak\Desktop\mykeibaAI\data\horse_result_html\2018105151.bin
[Skip] Existing bin file: C:\Users\yasak\Desktop\mykeibaAI\data\horse_result_html\2019106608.bin
[Skip] Existing bin file: C:\Users\yasak\Desktop\mykeibaAI\data\horse_result_html\2019110085.bin
[Skip] Existing bin file: C:\Users\yasak\Desktop\mykeibaAI\data\horse_result_html\2019101055.bin
[Skip] Existing bin file: C:\Users\yasak\Desktop\mykeibaAI\data\horse_result_html\2017101791.bin
[Skip] Existing bin file: C:\Users\yasak\Desktop\mykeibaAI\data\horse_result_html\2018101928.bin
[Skip] Existing bin file: C:\U

  0%|          | 24/13118 [00:01<14:35, 14.96it/s]

[Skip] Existing bin file: C:\Users\yasak\Desktop\mykeibaAI\data\horse_result_html\2017105910.bin
[Skip] Existing bin file: C:\Users\yasak\Desktop\mykeibaAI\data\horse_result_html\2018101175.bin


  0%|          | 29/13118 [00:06<1:16:20,  2.86it/s]

...

[Skip] Existing bin file: C:\Users\yasak\Desktop\mykeibaAI\data\horse_result_html\2021110123.bin
[Skip] Existing bin file: C:\Users\yasak\Desktop\mykeibaAI\data\horse_result_html\2021101213.bin
[Skip] Existing bin file: C:\Users\yasak\Desktop\mykeibaAI\data\horse_result_html\2021102456.bin
[Skip] Existing bin file: C:\Users\yasak\Desktop\mykeibaAI\data\horse_result_html\2021103770.bin
[Skip] Existing bin file: C:\Users\yasak\Desktop\mykeibaAI\data\horse_result_html\2021105506.bin
[Skip] Existing bin file: C:\Users\yasak\Desktop\mykeibaAI\data\horse_result_html\2021106245.bin
[Skip] Existing bin file: C:\Users\yasak\Desktop\mykeibaAI\data\horse_result_html\2021102522.bin
[Skip] Existing bin file: C:\Users\yasak\Desktop\mykeibaAI\data\horse_result_html\2021103039.bin
[Skip] Existing bin file: C:\Users\yasak\Desktop\mykeibaAI\data\horse_result_html\2021104501.bin
[Skip] Existing bin file: C:\Users\yasak\Desktop\mykeibaAI\data\horse_result_html\2021103904.bin
[Skip] Existing bin file: C:\U

 74%|███████▍  | 9742/13118 [2:12:30<05:43,  9.82it/s]

[Skip] Existing bin file: C:\Users\yasak\Desktop\mykeibaAI\data\horse_result_html\2021102026.bin
[Skip] Existing bin file: C:\Users\yasak\Desktop\mykeibaAI\data\horse_result_html\2021104714.bin


100%|██████████| 13118/13118 [2:12:32<00:00,  1.65it/s] 

[Skip] Existing bin file: C:\Users\yasak\Desktop\mykeibaAI\data\horse_result_html\2021106547.bin
[Skip] Existing bin file: C:\Users\yasak\Desktop\mykeibaAI\data\horse_result_html\2021106125.bin
[Skip] Existing bin file: C:\Users\yasak\Desktop\mykeibaAI\data\horse_result_html\2021105640.bin
[Skip] Existing bin file: C:\Users\yasak\Desktop\mykeibaAI\data\horse_result_html\2021101406.bin
[Skip] Existing bin file: C:\Users\yasak\Desktop\mykeibaAI\data\horse_result_html\2021101064.bin
[Skip] Existing bin file: C:\Users\yasak\Desktop\mykeibaAI\data\horse_result_html\2021105018.bin
[Skip] Existing bin file: C:\Users\yasak\Desktop\mykeibaAI\data\horse_result_html\2021104542.bin
[Skip] Existing bin file: C:\Users\yasak\Desktop\mykeibaAI\data\horse_result_html\2021101752.bin
[Skip] Existing bin file: C:\Users\yasak\Desktop\mykeibaAI\data\horse_result_html\2021105621.bin
[Skip] Existing bin file: C:\Users\yasak\Desktop\mykeibaAI\data\horse_result_html\2021106147.bin
[Skip] Existing bin file: C:\U

### 馬の過去成績テーブルの作成

In [14]:
# what: 各馬のbinファイルから馬の過去成績テーブルを抽出し、1つのテーブルに結合しpickleで保存
# for:  特徴量の抽出用
# in:   取得したhorse_id_list(.csv)とhtml(.bin)
# out:  結合されたresult_table(.pickle)

# 既存のhorse_result_tableに追加する場合はTrue、新規追加はFalse
add_table_bool = False

# 取得したhorse_idをcsvから読み込み
df = pd.read_csv(os.path.join(data_folder, horse_list_csv), header=None)

result_table_path = os.path.join(data_folder, horse_result_pkl)

if add_table_bool:
    # 既存pickleのhorse_idを確認
    existing_df = pd.read_pickle(result_table_path)
    existing_ids = set(existing_df["horse_id"].astype(str))
else:
    existing_df = pd.DataFrame() # 空のDataFrame
    existing_ids = set()

# 新しく解析するhorse_idだけを抽出
target_ids = [str(rid) for rid in df[0] if str(rid) not in existing_ids]

new_dfs = []
for horse_id in tqdm(target_ids, total=len(target_ids)):
    bin_path = os.path.join(data_folder, "horse_result_html", f"{horse_id}.bin")
    if not os.path.exists(bin_path):
        print(f"Missing bin file: {horse_id}")
        continue
    try:
        df_horse = parse_horse_html(bin_path)
        df_horse.insert(0, "horse_id", horse_id) # horse_idを先頭列に挿入
        new_dfs.append(df_horse)
    except Exception as e:
        print(f"Error fetching {horse_id}: {e}")
new_result_df = pd.concat(new_dfs, ignore_index=True)

# 追加するテーブルに前処理をしておく
new_result_df.drop(columns=["place"], inplace=True)
new_result_df["weather"] = new_result_df["weather"].apply(encoding_weather)
new_result_df["race_no"] = pd.to_numeric(new_result_df["race_no"], errors="coerce", downcast='float')
new_result_df["num_of_horses"] = pd.to_numeric(new_result_df["num_of_horses"], errors="coerce", downcast='float')
new_result_df["wakuban"] = pd.to_numeric(new_result_df["wakuban"], errors="coerce", downcast='float')
new_result_df["umaban"] = pd.to_numeric(new_result_df["umaban"], errors="coerce", downcast='float')
new_result_df["odds"] = pd.to_numeric(new_result_df["odds"], errors="coerce", downcast='float')
new_result_df["popularity"] = pd.to_numeric(new_result_df["popularity"], errors="coerce", downcast='float')
new_result_df["finish_position"] = pd.to_numeric(new_result_df["finish_position"], errors="coerce", downcast='float')
new_result_df["weight_carried"] = pd.to_numeric(new_result_df["weight_carried"], errors="coerce", downcast='float')
new_result_df[["course_state", "distance_length"]] = new_result_df["distance"].apply(lambda x: pd.Series(parse_encoding_distance(x)))
new_result_df.drop(columns=["distance"], inplace=True)
new_result_df["course_condition"] = new_result_df["course_condition"].apply(encoding_course_condition)
new_result_df["time"] = new_result_df["time"].apply(time_to_seconds)
new_result_df["margin"] = new_result_df["margin"].apply(parse_margin)
new_result_df["final_3f"] = new_result_df["final_3f"].apply(parse_margin)
new_result_df["prize_money"] = new_result_df["prize_money"].apply(parse_margin)
new_result_df[["passing_1st", "passing_2nd", "passing_3rd", "passing_4th"]] = new_result_df["passing"].apply(lambda x: pd.Series(parse_passing(x)))
new_result_df.drop(columns=["passing"], inplace=True)
new_result_df[["pace_1st", "pace_2nd"]] = new_result_df["pace"].apply(lambda x: pd.Series(parse_pace(x)))
new_result_df.drop(columns=["pace"], inplace=True)
new_result_df[["weight_horse","weight_diff"]] = new_result_df["weight"].apply(lambda x: pd.Series(parse_bodyweight(x)))
new_result_df.drop(columns=["weight"], inplace=True)

result_df = pd.concat([existing_df, new_result_df], ignore_index=True)
print(f"✅ 新規{len(new_result_df)}件を追加しました（合計 {len(result_df)} 件）")
result_table = result_df.to_pickle(result_table_path)

100%|██████████| 13118/13118 [39:55<00:00,  5.48it/s]


✅ 新規281469件を追加しました（合計 281469 件）


In [15]:
# 各列の型の確認
type_counts = {
    col: result_df[col].map(lambda x: str(type(x))).value_counts(sort=False)
    for col in result_df.columns
}

pd.DataFrame(type_counts)

,horse_id,race_date,weather,race_no,race_id,num_of_horses,wakuban,umaban,odds,popularity,finish_position,jockey_id,weight_carried,course_condition,time,margin,final_3f,prize_money,course_state,distance_length,passing_1st,passing_2nd,passing_3rd,passing_4th,pace_1st,pace_2nd,weight_horse,weight_diff
<class 'NoneType'>,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,24367.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
<class 'float'>,NaN,NaN,281469.0,281469.0,NaN,281469.0,281469.0,281469.0,281469.0,281469.0,281469.0,NaN,281469.0,281469.0,281469.0,281469.0,281469.0,281469.0,281469.0,281469.0,281469.0,281469.0,281469.0,281469.0,281469.0,281469.0,281469.0,281469.0
<class 'str'>,281469.0,281469.0,NaN,NaN,281469.0,NaN,NaN,NaN,NaN,NaN,NaN,257102.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
